In [1]:
DIR = "data_sets/"

In [2]:
# importing panda library
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from openpyxl import load_workbook
import numpy as np
import pandas as pd
from files.measures import n_1_imb_mean, n_3_imb_mean, tlcm, degOver, degIR, imbalance_ratio
import numpy as np 
from sklearn.utils import Bunch
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
from tqdm import tqdm

In [3]:
file_list = os.listdir(DIR)
len(file_list)
file_list[0:10]

['.ipynb_checkpoints',
 'australian.csv',
 'balance.csv',
 'bands.csv',
 'banknoteauthentication.csv',
 'bloodtransfusion.csv',
 'bupa.csv',
 'car_eval_34.csv',
 'car_eval_4.csv',
 'cleveland-0_vs_4.csv']

In [4]:
def unique(list1):
 
    # initialize a null list
    unique_list = []
 
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print(x)

In [5]:
names = []
filename = []
error_files = []
good_files = []

for i in range(len(file_list)):
    if file_list[i] == ".ipynb_checkpoints":
        pass
    else:
        df = pd.read_csv(f"{DIR}/{file_list[i]}")
        names.append(df.columns[-1])
        filename.append(file_list[i])

for i in range(len(names)):
    if names[i] in ["Output", "Output", "outcome", "output"]:
        good_files.append(filename[i])
    else:
        error_files.append(filename[i])     
        
error_files

[]

In [6]:
def load_dataset(dataset) -> Bunch: 
    dataset=np.array(dataset)
    X = dataset[:,0:-1]
    y = dataset[:,-1]
    return Bunch(data=np.array(X), target=y)

In [7]:
def result_df(dataset_): 
    n_1_score = n_1_imb_mean(dataset_) 
    n_3_score = n_3_imb_mean(dataset_)
    tlcm_score = tlcm(dataset_)   
    degOver_score = degOver(dataset_)
    degIR_score = degIR(dataset_)
    ir_score = imbalance_ratio(dataset_)
    return  n_1_score, n_3_score,tlcm_score, degOver_score, degIR_score, ir_score

In [8]:
def excel_manipulator(filename):
    new_df = pd.read_csv(f"final_result_{filename}.csv")
    new_df.to_excel(f"final_result_{filename}.xlsx")
    new_df = pd.read_excel(f"final_result_{filename}.xlsx")
    new_df = new_df.drop(new_df.columns[0], axis=1)
    new_df.to_excel(f"final_result_{filename}_excel.xlsx")
    workbook = load_workbook(filename=f"final_result_{filename}_excel.xlsx")
    sheet = workbook.active
    sheet["B1"].value = f"{filename}"
    workbook.save(f'{filename}_modified.xlsx')
    df = pd.read_excel(f'{filename}_modified.xlsx')
    df = df.drop(df.columns[0], axis=1)
    df.to_excel("{filename}_modified.xlsx", index = False)
    os.remove(f"final_result_{filename}_excel.xlsx")
    os.remove(f"final_result_{filename}.xlsx")
    os.remove(f"final_result_{filename}.csv")

In [9]:
def process(filename):
    dataframe=pd.read_csv(f"{DIR}/{filename}")
    
    column_name = dataframe.columns[-1]
    y=dataframe[f'{column_name}']
    X=dataframe.drop([f'{column_name}'], axis=1)
    complexity_measure_train={}
    complexity_measure_test={}
    skf = StratifiedKFold(n_splits=5, random_state=55, shuffle=True)
    
    
    for i, (train_index, test_index) in enumerate(skf.split(dataframe, y)):
      train_=dataframe.loc[train_index,:]
      with open(f"{filename}_train_indices.txt", "w") as output:
        for j in range(5):
           output.write(str(train_index))

      dataset_train=load_dataset(train_)
      complexity_measure_train["train_"+ str(i)]={}
      n_1_score, n_3_score,tlcm_score,degOver_score, degIR_score, ir_score=result_df(dataset_train)
      complexity_measure_train["train_"+ str(i)]["n_1_score"]=n_1_score
      complexity_measure_train["train_"+ str(i)]["n_3_score"]=n_3_score
      complexity_measure_train["train_"+ str(i)]["tlcm_score"]=tlcm_score
      complexity_measure_train["train_"+ str(i)]["degOver_score"]=degOver_score
      complexity_measure_train["train_"+ str(i)]["degIR_score"]=degIR_score
      complexity_measure_train["train_"+ str(i)]["IR_score"]=ir_score

      test_= dataframe.loc[test_index,:]
      with open(f"{filename}_test_indices.txt", "w") as output:
        for j in range(5):
          output.write(str(test_index))

      dataset_test=load_dataset(test_)
      complexity_measure_test["test_"+ str(i)]={}
      n_1_score, n_3_score,tlcm_score,degOver_score, degIR_score, ir_score=result_df(dataset_test)
      complexity_measure_test["test_"+ str(i)]["n_1_score"]=n_1_score
      complexity_measure_test["test_"+ str(i)]["n_3_score"]=n_3_score
      complexity_measure_test["test_"+ str(i)]["tlcm_score"]=tlcm_score
      complexity_measure_test["test_"+ str(i)]["degOver_score"]=degOver_score
      complexity_measure_test["test_"+ str(i)]["degIR_score"]=degIR_score
      complexity_measure_test["test_"+ str(i)]["IR_score"]=ir_score
      i+=1
    
    df_train_set= pd.DataFrame.from_dict(complexity_measure_train)  
    df_train_set.transpose()
    
    plt.figure(figsize=(7,5))
    sns.boxplot(data=df_train_set.transpose(), whis=[0, 100], width=.6, palette="vlag")
    sns.stripplot(data=df_train_set.transpose(),size=4,color=".3", linewidth=0)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    df_test_set= pd.DataFrame.from_dict(complexity_measure_test)

    plt.figure(figsize=(8,5))
    sns.boxplot(data=df_test_set.transpose(), whis=[0, 100], width=.6, palette="vlag")
    sns.stripplot(data=df_test_set.transpose(),size=4,color=".3", linewidth=0)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    result= df_train_set.transpose().mean()
    final_result= pd.DataFrame(result, columns=["Train_Mean_Score"])
    final_result["Train_standard_deviation"]=df_train_set.transpose().std()
    final_result["Test_Mean_Score"]=df_test_set.transpose().mean()
    final_result["Test_standard_deviation"]=df_test_set.transpose().std()
    
    final_result.transpose().to_csv(f"final_result_{filename}")
    
    final_result.drop(['IR_score'], inplace=True)
    fig, ax = plt.subplots()
    ax.bar(final_result.index, final_result["Train_Mean_Score"], yerr=final_result['Train_standard_deviation'], align='center', alpha=0.8, ecolor='black', capsize=12)
    ax.set_xticks(final_result.index)
    ax.set_ylabel('Range')
    ax.set_title(f'{filename.replace(".csv","")}')
    ax.yaxis.grid(True)
    plt.tight_layout()
    plt.savefig(f'{filename.replace(".csv","")}_Train_bar_plot_with_error_bars.png')

    
    fig, ax = plt.subplots()
    ax.bar(final_result.index, final_result["Test_Mean_Score"], yerr=final_result['Test_standard_deviation'], align='center', alpha=0.8, ecolor='black', capsize=12)
    ax.set_xticks(final_result.index)
    ax.set_title(f'{filename.replace(".csv","")}')
    ax.set_ylabel('Range')
    ax.yaxis.grid(True)
    plt.tight_layout()
    plt.savefig(f'{filename.replace(".csv","")}_Test_bar_plot_with_error_bars.png')
    
    
    filename = filename[:-4]
    excel_manipulator(filename)

        

    

In [ ]:
for i in tqdm(range(len(good_files))):
     if file_list[i] not in ('.ipynb_checkpoints'):
            process(file_list[i])

  2%|▏         | 3/143 [01:03<53:36, 22.97s/it]